# Source Code Changes

This notebook generates the table for source code changes.


In [1]:
%reload_ext autoreload
%autoreload 2

In [11]:
import pandas as pd

from depsurf import DepKind, DiffResult, IssueEnum, VersionGroup
from utils import GRAY_DASH, load_pkl, save_latex, mini_bar, multirow

data: DiffResult = load_pkl("src")

KINDS = [
    DepKind.FUNC,
    # DepKind.LSM,
    DepKind.STRUCT,
    DepKind.TRACEPOINT,
]

ISSUES = {
    IssueEnum.OLD: r"\ \ \ # \ \ ",
    IssueEnum.ADD: r"$+$%",
    IssueEnum.REMOVE: r"$-$%",
    IssueEnum.CHANGE: r"$\Delta$%",
}

COLORS = {
    IssueEnum.ADD: "add",
    IssueEnum.REMOVE: "remove",
    IssueEnum.CHANGE: "change",
}

GROUPS = {
    VersionGroup.LTS: "LTS (2 yr)",
    VersionGroup.REGULAR: "Regular Releases (6 mo)",
}


table = {}


def format_val(kind, val) -> str:
    if kind == DepKind.FUNC:
        return f"{val / 1000:.0f}k"
    elif kind == DepKind.STRUCT:
        return f"{val / 1000:.1f}k"
    return str(val)

def format_percent(num):
    pct = num * 100
    if pct < 0.5:
        return f"{pct:.1f}"
    return f"{pct:.0f}"

max_percent = {
    (kind, issue): max(
        pair_result.kind_results[kind].issues[issue]
        / pair_result.kind_results[kind].issues[IssueEnum.OLD]
        for group, group_result in data.iter_groups()
        for pair, pair_result in group_result.iter_pairs()
    )
    for kind in KINDS
    for issue in ISSUES
}


for group, group_result in data.iter_groups():
    if group not in GROUPS:
        continue

    for pair, pair_result in group_result.iter_pairs():
        col = {}
        for kind, kind_result in pair_result.iter_kinds():
            if kind not in KINDS:
                continue
            old = kind_result.issues[IssueEnum.OLD]
            for issue, count in kind_result.iter_issues():
                if issue not in ISSUES:
                    continue

                if issue == IssueEnum.OLD:
                    text = format_val(kind, count)
                else:
                    if count == 0:
                        text = GRAY_DASH
                    else:
                        text = mini_bar(
                            text=format_percent(count / old),
                            percent=count / old / max_percent[(kind, issue)],
                            total_width=0.55,
                            color=COLORS[issue],
                        )
                    text = multirow(text)
                col[(kind, ISSUES[issue])] = text

        v1 = group.to_str(pair.v1)
        v2 = group.to_str(pair.v2)

        group_name = GROUPS[group]
        table[(group_name, v1)] = col
        if pair.v2 == list(group)[-1]:
            col = {k: "" for k in col.keys()}
            for kind, key in col.keys():
                if key == ISSUES[IssueEnum.OLD]:
                    count = pair_result.kind_results[kind].issues[IssueEnum.NEW]
                    col[(kind, key)] = format_val(kind, count)
            table[(group_name, v2)] = col


df = pd.DataFrame(table).T
latex = df.to_latex(multicolumn_format="c|", column_format=r"cc|rrrr|rrrr|rrrr")
save_latex(latex, "src")
df

[ utils_pickle.py:18 ] INFO: Loding src from /Users/szhong/Code/DepSurf/output/src.pkl
[  utils_latex.py:118] INFO: Saved src to /Users/szhong/Code/DepSurf/paper/tabs/src.tex


Function                                                                                                                                                                                                         Struct                                                                                                                                                                                                       Tracepoint                                                                                                                                                                                                     
                             \ \ \ # \ \                                                            $+$%                                                              $-$%                                                        $\Delta$% \ \ \ # \ \                                                            $+$%                                                               $-$%                                                         $\Delta$% \ \ \ # \ \                                                            $+$%                                                               $-$%                                                          $\Delta$%
LTS (2 yr)              4.4           36k  \multirow[c]{2}{*}{\colorbox{add}{\makebox[{0.55cm}][r]{24}}}   \multirow[c]{2}{*}{\colorbox{remove}{\makebox[{0.39cm}][r]{7}}}  \multirow[c]{2}{*}{\colorbox{change}{\makebox[{0.53cm}][r]{5}}}         6.2k  \multirow[c]{2}{*}{\colorbox{add}{\makebox[{0.55cm}][r]{24}}}    \multirow[c]{2}{*}{\colorbox{remove}{\makebox[{0.40cm}][r]{4}}}  \multirow[c]{2}{*}{\colorbox{change}{\makebox[{0.55cm}][r]{18}}}          502  \multirow[c]{2}{*}{\colorbox{add}{\makebox[{0.55cm}][r]{39}}}    \multirow[c]{2}{*}{\colorbox{remove}{\makebox[{0.55cm}][r]{5}}}    \multirow[c]{2}{*}{\colorbox{change}{\makebox[{0.27cm}][r]{8}}}
                        4.15          42k  \multirow[c]{2}{*}{\colorbox{add}{\makebox[{0.49cm}][r]{21}}}   \multirow[c]{2}{*}{\colorbox{remove}{\makebox[{0.41cm}][r]{8}}}  \multirow[c]{2}{*}{\colorbox{change}{\makebox[{0.42cm}][r]{4}}}         7.3k  \multirow[c]{2}{*}{\colorbox{add}{\makebox[{0.43cm}][r]{19}}}    \multirow[c]{2}{*}{\colorbox{remove}{\makebox[{0.36cm}][r]{4}}}  \multirow[c]{2}{*}{\colorbox{change}{\makebox[{0.47cm}][r]{15}}}          675  \multirow[c]{2}{*}{\colorbox{add}{\makebox[{0.20cm}][r]{15}}}    \multirow[c]{2}{*}{\colorbox{remove}{\makebox[{0.36cm}][r]{3}}}    \multirow[c]{2}{*}{\colorbox{change}{\makebox[{0.28cm}][r]{8}}}
                        5.4           48k  \multirow[c]{2}{*}{\colorbox{add}{\makebox[{0.52cm}][r]{22}}}  \multirow[c]{2}{*}{\colorbox{remove}{\makebox[{0.55cm}][r]{10}}}  \multirow[c]{2}{*}{\colorbox{change}{\makebox[{0.45cm}][r]{5}}}         8.4k  \multirow[c]{2}{*}{\colorbox{add}{\makebox[{0.39cm}][r]{17}}}    \multirow[c]{2}{*}{\colorbox{remove}{\makebox[{0.55cm}][r]{6}}}  \multirow[c]{2}{*}{\colorbox{change}{\makebox[{0.47cm}][r]{16}}}          752  \multirow[c]{2}{*}{\colorbox{add}{\makebox[{0.19cm}][r]{14}}}    \multirow[c]{2}{*}{\colorbox{remove}{\makebox[{0.55cm}][r]{5}}}   \multirow[c]{2}{*}{\colorbox{change}{\makebox[{0.55cm}][r]{16}}}
                        5.15          54k  \multirow[c]{2}{*}{\colorbox{add}{\makebox[{0.52cm}][r]{23}}}   \multirow[c]{2}{*}{\colorbox{remove}{\makebox[{0.42cm}][r]{8}}}  \multirow[c]{2}{*}{\colorbox{change}{\makebox[{0.55cm}][r]{6}}}         9.3k  \multirow[c]{2}{*}{\colorbox{add}{\makebox[{0.38cm}][r]{16}}}    \multirow[c]{2}{*}{\colorbox{remove}{\makebox[{0.35cm}][r]{4}}}  \multirow[c]{2}{*}{\colorbox{change}{\makebox[{0.49cm}][r]{16}}}          818  \multirow[c]{2}{*}{\colorbox{add}{\makebox[{0.26cm}][r]{18}}}    \multirow[c]{2}{*}{\colorbox{remove}{\makebox[{0.51cm}][r]{4}}}   \multirow[c]{2}{*}{\colorbox{change}{\makebox[{0.49cm}][r]{14}}}
                        6.8           62k                                                              